#Day63 2025-12-10
## Extract

In [0]:
# NOT requuired in databricks notebook since spark session is already created and available
from pyspark.sql import SparkSession
spark1 = SparkSession.builder.appName("Car Demo").getOrCreate()
print(id(spark) , " - ID of spark created and given to us by Databricks")
print(id(spark1), " - ID of spark created by us; its not created but get from existing instance by SparkSession.builder.getOrCreate() method")
print("Both IDs will be same")